# Phase 1: 数据集整理 (DataSet exposing)

## 概述 Overview

本Notebook文件基于Phase1脚本修改而来，目标为按照一定比例，融合不同专利的数据集，形成测试集和验证集，并且将测试集合和验证集合命名为mixture_{train/test}.csv和npz文件
筛选操作也是同样做处理

**主要步骤：**
* 4268数据集合完全划分到训练集中
* 剩下的所有数据7为训练集，3为测试集合

**生成结果**
* csv文件（没有processed后缀，存放在data/mixture/csv文件夹下方）
* npz文件（没有processed后缀，存放在data/mixture/feature下方）

**输入**: `data/raw/*.csv` - 原始数据文件  
**输出**: 
- `data/mixture/csv/*.csv` - 添加分子特征后的CSV  
- `data/mixture/feature/*.npz` - RDKit特征矩阵  
- `outputs/figures/phase1/*.png` - 质量验证图表（这个暂时没有） 

**总样本数**: 1,931 → 932 (48.3% retention)

---

## 1. 环境检查与导入 Environment Setup

In [19]:
# 环境检查
import sys
from pathlib import Path

# 添加项目根目录到路径
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# 核心库导入
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# 项目模块导入
from feature_extraction import PeptideFeaturizer
from feature_extraction.utils import (
    get_csv_files, load_csv_safely, extract_molecular_features,
    convert_label_to_minutes, save_features_to_npz
)

# 设置显示选项
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

print("✓ 所有库已成功导入")
print(f"✓ 项目根目录: {project_root}")




✓ 所有库已成功导入
✓ 项目根目录: d:\RA\feature_extraction


## 2. 参数配置区 Configuration

**⚙️ 根据您的需求修改以下参数**

In [ ]:
# ============== 参数配置区 ==============
# 用户可根据需要修改以下参数

CONFIG = {
    # 读入数据路径
    'raw_dir': project_root / 'data' / 'raw',
    # csv输出路径
    'processed_dir': project_root / 'data' / 'mixture' / 'csv',
    'features_dir': project_root / 'data' / 'mixture' / 'features',
    'figures_dir': project_root / 'data' / 'mixture'/ 'figures' / 'phase1',
    
    # 特征提取参数
    'morgan_bits': 1024, #1024,     # Morgan指纹位数（这里临时修改为0，关闭特征提取）
    'avalon_bits': 512,      # Avalon指纹位数512
    # （放在外面配置了）'use_avalon': True ,      # 是否使用Avalon指纹（需RDKit支持） 

    # 可视化参数
    'dpi': 300,              # 图像分辨率
    'format': 'png',         # 图像格式 (png/pdf/svg)
    'display_plots': True,   # 是否在notebook中显示关键图表
    'max_display_plots': 3,  # 最多显示几个图表
}

# 追加配置
CONFIG['use_avalon'] = True if CONFIG['avalon_bits'] > 0 else False  # 是否使用Avalon指纹（需RDKit支持）

# 创建输出目录
CONFIG['processed_dir'].mkdir(parents=True, exist_ok=True)
CONFIG['features_dir'].mkdir(parents=True, exist_ok=True)
CONFIG['figures_dir'].mkdir(parents=True, exist_ok=True)

print("配置参数:")
for key, value in CONFIG.items():
    if isinstance(value, Path):
        print(f"  {key}: {value.relative_to(project_root) if value.is_relative_to(project_root) else value}")
    else:
        print(f"  {key}: {value}")

配置参数:
  raw_dir: data\raw
  processed_dir: data\mixture\csv
  features_dir: data\mixture\features
  figures_dir: data\mixture\figures\phase1
  morgan_bits: 1024
  avalon_bits: 512
  use_avalon: True
  dpi: 300
  format: png
  display_plots: True
  max_display_plots: 3


## 3. 步骤 1: 添加分子特征 Add Molecular Features

为每个SMILES分子添加结构特征并转换标签格式。

**新增列**:
- `is_dimer`: 是否为二聚体 (bool)
- `is_cyclic`: 是否含环状结构 (bool)
- `has_disulfide_bond`: 是否含二硫键 (bool)
- `SIF_minutes`: SIF半衰期（分钟）
- `SGF_minutes`: SGF半衰期（分钟）

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

def process_and_merge_datasets(file_paths, full_train_names, train_ratio=0.7, message:str=""):
    """
    file_paths: list, 所有数据集的文件路径列表 (例如 ['data1.csv', 'patents.csv', ...])
    full_train_names: list, 需要全部作为训练集的文件名关键词 (例如 ['patents'])
    train_ratio: float, 其他数据集的训练集比例
    """
    train_list = []
    test_list = []

    for path in file_paths:
        # 1. 获取文件名并读取原始数据
        file_name = path
        csv_path = CONFIG['raw_dir'] / path
        print(f"正在处理: {file_name}")
        df = pd.read_csv(csv_path)
        
        # ------------------ 原有预处理逻辑开始 ------------------
        # 提取分子特征
        feature_records = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"处理 {file_name}", leave=False):
            smiles = row["SMILES"]
            features = extract_molecular_features(smiles)
            feature_records.append(features)
        
        # 添加特征列
        feature_df = pd.DataFrame(feature_records)
        df = pd.concat([df, feature_df], axis=1)
        
        # 转换标签到分钟
        sif_col = "SIF_class" if "SIF_class" in df.columns else None
        sgf_col = "SGF_class" if "SGF_class" in df.columns else None
        
        if sif_col:
            df["SIF_minutes"] = df[sif_col].apply(convert_label_to_minutes)
        else:
            df["SIF_minutes"] = -1
        
        if sgf_col:
            df["SGF_minutes"] = df[sgf_col].apply(convert_label_to_minutes)
        else:
            df["SGF_minutes"] = -1
        
        # 过滤双标签缺失的样本
        mask_both_missing = (df["SIF_minutes"] == -1) & (df["SGF_minutes"] == -1)
        df_processed = df[~mask_both_missing].copy()
        # ------------------ 原有预处理逻辑结束 ------------------

        # 2. 划分与合并逻辑
        # 判断当前处理好的 df_processed 应该往哪儿走
        is_full_train = any(name in file_name for name in full_train_names)
        
        if is_full_train:
            print(f"--- 模式: {file_name} 全量加入训练集")
            train_list.append(df_processed)
        else:
            # 按照 7:3 划分处理后的数据
            if len(df_processed) < 2:
                print(f"--- 警告: {file_name} 数据量过小，全量计入训练集")
                train_list.append(df_processed)
            else:
                print(f"--- 模式: {file_name} 按 7:3 划分")
                train_df, test_df = train_test_split(
                    df_processed, 
                    train_size=train_ratio, 
                    random_state=42
                )
                train_list.append(train_df)
                test_list.append(test_df)
        
    # 合并所有数据
    final_train = pd.concat(train_list, ignore_index=True)
    final_test = pd.concat(test_list, ignore_index=True) if test_list else pd.DataFrame()

    # 输出结果
    final_train.to_csv(CONFIG["processed_dir"]/f'mixture_train_{message}.csv', index=False)
    final_test.to_csv(CONFIG["processed_dir"]/f'mixture_test_{message}.csv', index=False)

    print("\n" + "="*30)
    print(f"合并完成！")
    print(f"最终训练集样本数: {len(final_train)}")
    print(f"最终测试集样本数: {len(final_test)}")
    print("="*30)

# --- 使用示例 ---


# 执行：批量处理所有CSV文件
csv_files = list(CONFIG['raw_dir'].glob('*.csv'))
csv_files_names = [file.name for file in csv_files]
print(f"找到 {len(csv_files)} 个CSV文件\n")

# 指定特定数据集加入训练集合
special_datasets = ['US9624268.csv'] 
process_and_merge_datasets(csv_files_names, special_datasets)
# 保留样本数目为932就对了


找到 5 个CSV文件

正在处理: sif_sgf_second.csv


处理 sif_sgf_second.csv:   0%|          | 0/897 [00:00<?, ?it/s]

--- 模式: sif_sgf_second.csv 按 7:3 划分
正在处理: US20140294902A1.csv


处理 US20140294902A1.csv:   0%|          | 0/5 [00:00<?, ?it/s]

--- 模式: US20140294902A1.csv 按 7:3 划分
正在处理: US9624268.csv


...

--- 模式: US9624268.csv 全量加入训练集
正在处理: US9809623B2.csv


处理 US9809623B2.csv:   0%|          | 0/80 [00:00<?, ?it/s]

--- 模式: US9809623B2.csv 按 7:3 划分
正在处理: WO2017011820A2.csv


处理 WO2017011820A2.csv:   0%|          | 0/174 [00:00<?, ?it/s]

--- 模式: WO2017011820A2.csv 按 7:3 划分

合并完成！
最终训练集样本数: 690
最终测试集样本数: 242


## 4. 步骤 2: 提取RDKit特征 Extract RDKit Features

从处理后的CSV中提取分子特征向量，保存为NPZ格式。

**特征类型**:
- QED属性 (8维)
- 物理化学描述符 (11维)
- Gasteiger电荷统计 (5维)
- Morgan指纹 (1024维)
- Avalon指纹 (512维, 可选)

In [22]:
from pathlib import Path
def extract_rdkit_features(csv_path: Path, output_dir: Path, featurizer):
    """
    从CSV提取RDKit特征并保存为NPZ
    
    Args:
        csv_path: 输入CSV文件路径
        output_dir: 输出目录
        featurizer: PeptideFeaturizer实例
    
    Returns:
        dict: 统计信息
    """
    # 加载CSV
    df, _ = load_csv_safely(csv_path, required_columns=["id", "SMILES", "SIF_minutes", "SGF_minutes"])
    if df is None:
        return {"error": "Failed to load CSV"}
    
    X = []
    y_sif = []
    y_sgf = []
    ids = []
    valid_count = 0
    
    # 提取特征
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"提取特征 {csv_path.name}", leave=False):
        smiles = str(row["SMILES"])
        features, success = featurizer.featurize(smiles)
        
        if success and features is not None:
            X.append(features)
            y_sif.append(int(row["SIF_minutes"]) if not pd.isna(row["SIF_minutes"]) else -1)
            y_sgf.append(int(row["SGF_minutes"]) if not pd.isna(row["SGF_minutes"]) else -1)
            ids.append(str(row["id"]))
            valid_count += 1
    
    # 转换为NumPy数组
    X = np.array(X, dtype=np.float32)
    y_sif = np.array(y_sif, dtype=np.int32)
    y_sgf = np.array(y_sgf, dtype=np.int32)
    ids = np.array(ids, dtype=object)
    feature_names = featurizer.get_feature_names()
    
    # 保存NPZ
    output_path = output_dir / csv_path.name.replace('.csv', '.npz')
    np.savez_compressed(
        output_path,
        X=X,
        y_sif=y_sif,
        y_sgf=y_sgf,
        ids=ids,
        feature_names=feature_names,
    )
    
    return {
        "file": csv_path.name,
        "total_samples": len(df),
        "valid_samples": valid_count,
        "feature_dim": X.shape[1],
        "output_path": output_path,
    }

# 初始化特征提取器
featurizer = PeptideFeaturizer(
    morgan_bits=CONFIG['morgan_bits'],
    avalon_bits=CONFIG['avalon_bits'],
    use_avalon=CONFIG['use_avalon']
)

print(f"特征提取器配置:")
print(f"  Morgan指纹: {CONFIG['morgan_bits']} bits")
print(f"  Avalon指纹: {CONFIG['avalon_bits']} bits (启用: {CONFIG['use_avalon']})")
print(f"  预计总特征维度: {featurizer.n_features}\n")

# 执行：批量提取特征
processed_csvs = list(CONFIG['processed_dir'].glob('mixture_*.csv'))
print(f"找到 {len(processed_csvs)} 个处理后的CSV文件\n")

feature_stats = []
for csv_file in processed_csvs:
    stats = extract_rdkit_features(csv_file, CONFIG['features_dir'], featurizer)
    if "error" not in stats:
        feature_stats.append(stats)
        print(f"✓ {stats['file']}: {stats['valid_samples']} samples, {stats['feature_dim']} features")

# 汇总
feat_summary_df = pd.DataFrame(feature_stats)
print(f"\n{'='*60}")
print("特征提取总结:")
print(f"  总样本数: {feat_summary_df['valid_samples'].sum()}")
print(f"  特征维度: {feat_summary_df['feature_dim'].iloc[0]}")
print(f"{'='*60}\n")

display(feat_summary_df[['file', 'total_samples', 'valid_samples', 'feature_dim']])

特征提取器配置:
  Morgan指纹: 1024 bits
  Avalon指纹: 512 bits (启用: True)
  预计总特征维度: 1560

找到 2 个处理后的CSV文件



提取特征 mixture_test_.csv:   0%|          | 0/242 [00:00<?, ?it/s]

[23:35:35] DEPRECATION WARNING: please use MorganGenerator
[23:35:35] DEPRECATION WARNING: please use MorganGenerator
[23:35:35] DEPRECATION WARNING: please use MorganGenerator
[23:35:35] DEPRECATION WARNING: please use MorganGenerator
[23:35:35] DEPRECATION WARNING: please use MorganGenerator
[23:35:36] DEPRECATION WARNING: please use MorganGenerator
[23:35:36] DEPRECATION WARNING: please use MorganGenerator
[23:35:36] DEPRECATION WARNING: please use MorganGenerator
[23:35:36] DEPRECATION WARNING: please use MorganGenerator
[23:35:37] DEPRECATION WARNING: please use MorganGenerator
[23:35:37] DEPRECATION WARNING: please use MorganGenerator
[23:35:37] DEPRECATION WARNING: please use MorganGenerator
[23:35:37] DEPRECATION WARNING: please use MorganGenerator
[23:35:37] DEPRECATION WARNING: please use MorganGenerator
[23:35:38] DEPRECATION WARNING: please use MorganGenerator
[23:35:38] DEPRECATION WARNING: please use MorganGenerator
[23:35:38] DEPRECATION WARNING: please use MorganGenerat

✓ mixture_test_.csv: 241 samples, 1560 features


提取特征 mixture_train_.csv:   0%|          | 0/690 [00:00<?, ?it/s]

[23:36:07] DEPRECATION WARNING: please use MorganGenerator
[23:36:07] DEPRECATION WARNING: please use MorganGenerator
[23:36:07] DEPRECATION WARNING: please use MorganGenerator
[23:36:07] DEPRECATION WARNING: please use MorganGenerator
[23:36:07] DEPRECATION WARNING: please use MorganGenerator
[23:36:07] DEPRECATION WARNING: please use MorganGenerator
[23:36:07] DEPRECATION WARNING: please use MorganGenerator
[23:36:07] DEPRECATION WARNING: please use MorganGenerator
[23:36:08] DEPRECATION WARNING: please use MorganGenerator
[23:36:08] DEPRECATION WARNING: please use MorganGenerator
[23:36:08] DEPRECATION WARNING: please use MorganGenerator
[23:36:08] DEPRECATION WARNING: please use MorganGenerator
[23:36:09] DEPRECATION WARNING: please use MorganGenerator
[23:36:09] DEPRECATION WARNING: please use MorganGenerator
[23:36:09] DEPRECATION WARNING: please use MorganGenerator
[23:36:09] DEPRECATION WARNING: please use MorganGenerator
[23:36:09] DEPRECATION WARNING: please use MorganGenerat

✓ mixture_train_.csv: 686 samples, 1560 features

特征提取总结:
  总样本数: 927
  特征维度: 1560



[23:37:23] DEPRECATION WARNING: please use MorganGenerator
[23:37:23] DEPRECATION WARNING: please use MorganGenerator


,file,total_samples,valid_samples,feature_dim
0,mixture_test_.csv,242,241,1560
1,mixture_train_.csv,690,686,1560


## 5. 步骤 3: 质量验证可视化 Quality Validation

生成可视化图表验证Phase 1的数据处理质量。

In [ ]:
# 由于前面的代码修改，这里暂时先不用了。。。。。。、
# 后续有机会再加上统计

## 6. 结果总结 Summary

Phase 1 数据转化已完成！

In [ ]:
print("="*70)
print("Phase 1: 数据转化 - 执行完毕")
print("="*70)

print("\n📁 生成的文件:")
print(f"\n  1. 处理后的CSV ({len(list(CONFIG['processed_dir'].glob('*.csv')))} 个文件):")
for f in sorted(CONFIG['processed_dir'].glob('*_processed.csv')):
    print(f"     - {f.name}")

print(f"\n  2. 特征NPZ文件 ({len(list(CONFIG['features_dir'].glob('*.npz')))} 个文件):")
for f in sorted(CONFIG['features_dir'].glob('*.npz')):
    size_mb = f.stat().st_size / 1024 / 1024
    print(f"     - {f.name} ({size_mb:.2f} MB)")

print(f"\n  3. 验证图表 ({len(list(CONFIG['figures_dir'].glob('*.png')))} 个文件):")
for f in sorted(CONFIG['figures_dir'].glob('*.png')):
    print(f"     - {f.name}")

print("\n📊 处理统计:")